In [115]:
cd ~/Flux_Eddypro/evaluasi

C:\Users\ahlil\Flux_Eddypro\evaluasi


In [116]:
pip install xarray netCDF4


Note: you may need to restart the kernel to use updated packages.


In [117]:
import xarray as xr


In [118]:
import xarray as xr

# List of NetCDF files to be merged
nc_files = ['data_2017.nc', 'data_2019.nc','data_2018.nc']

# Open and merge the NetCDF files into a single dataset
combined_dataset = xr.open_mfdataset(nc_files, combine='by_coords')

# Display the merged dataset
combined_dataset


<xarray.Dataset>
Dimensions:    (longitude: 81, latitude: 141, time: 5088)
Coordinates:
  * longitude  (longitude) float32 110.0 110.2 110.5 110.8 ... 129.5 129.8 130.0
  * latitude   (latitude) float32 25.0 24.75 24.5 24.25 ... -9.5 -9.75 -10.0
  * time       (time) datetime64[ns] 2017-12-01 ... 2020-12-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(744, 141, 81), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(744, 141, 81), meta=np.ndarray>
    sst        (time, latitude, longitude) float32 dask.array<chunksize=(744, 141, 81), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-24 12:02:11 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [119]:
import xarray as xr

# Path to the NetCDF file you want to open
file_path = 'output Jawa.nc'

# Open the NetCDF file
dataset = xr.open_dataset(file_path)

# Display the dataset
print(dataset)


<xarray.Dataset>
Dimensions:        (Stasiun: 156, Tanggal: 2557)
Coordinates:
  * Stasiun        (Stasiun) object '06-02008' '06-02010' ... 'STA5132'
  * Tanggal        (Tanggal) datetime64[ns] 2016-01-01 2016-01-02 ... 2022-12-31
Data variables:
    ID Stasiun     (Stasiun) object ...
    Nama Stasiun   (Stasiun) object ...
    Provinsi       (Stasiun) object ...
    lat            (Stasiun) float64 ...
    lon            (Stasiun) float64 ...
    ws_avg         (Stasiun, Tanggal) float64 ...
    ws_max         (Stasiun, Tanggal) float64 ...
    ws_avg nonlts  (Stasiun, Tanggal) float64 ...
    ws_max nonlts  (Stasiun, Tanggal) float64 ...
    ltsmask        (Stasiun, Tanggal) float64 ...
    r2             (Stasiun) float64 ...


In [120]:
# Select the DJF period for the years 2017 and 2018
djf_dataset = dataset.sel(Tanggal=((dataset.Tanggal.dt.month == 12) & (dataset.Tanggal.dt.year == 2017)) | 
                                     ((dataset.Tanggal.dt.month.isin([1, 2])) & (dataset.Tanggal.dt.year == 2018)) )


# Display the sliced dataset
print(djf_dataset)


<xarray.Dataset>
Dimensions:        (Stasiun: 156, Tanggal: 90)
Coordinates:
  * Stasiun        (Stasiun) object '06-02008' '06-02010' ... 'STA5132'
  * Tanggal        (Tanggal) datetime64[ns] 2017-12-01 2017-12-02 ... 2018-02-28
Data variables:
    ID Stasiun     (Stasiun) object ...
    Nama Stasiun   (Stasiun) object ...
    Provinsi       (Stasiun) object ...
    lat            (Stasiun) float64 ...
    lon            (Stasiun) float64 ...
    ws_avg         (Stasiun, Tanggal) float64 ...
    ws_max         (Stasiun, Tanggal) float64 ...
    ws_avg nonlts  (Stasiun, Tanggal) float64 ...
    ws_max nonlts  (Stasiun, Tanggal) float64 ...
    ltsmask        (Stasiun, Tanggal) float64 ...
    r2             (Stasiun) float64 ...


In [121]:
import xarray as xr
import pandas as pd
import numpy as np

# Assuming combined_dataset is already loaded as an xarray Dataset
# Select data at the given coordinates (110, 0) for all time
selected_data = combined_dataset.sel(longitude=110, latitude=0, method='nearest')

# Convert the data to a pandas DataFrame
df = selected_data[['u10', 'v10']].to_dataframe().reset_index()

# Calculate wind speed
df['ws_avg'] = np.sqrt(df['u10']**2 + df['v10']**2)

# Set 'time' as the index for resampling
df.set_index('time', inplace=True)

# Resample to daily frequency and calculate the mean wind speed
daily_mean_df = df.resample('D').mean().reset_index()

# Display the first few rows of the daily mean DataFrame
print(daily_mean_df[['time', 'u10','v10','ws_avg']])


           time       u10       v10    ws_avg
0    2017-12-01  0.780274  0.523236  1.379070
1    2017-12-02  0.608865  0.900866  1.446645
2    2017-12-03  0.373808  0.317415  0.885489
3    2017-12-04  0.349532 -0.079821  1.150266
4    2017-12-05  0.085431 -0.357639  0.944595
...         ...       ...       ...       ...
1122 2020-12-27  0.375574 -0.606301  1.098039
1123 2020-12-28  0.514156  0.165423  1.149944
1124 2020-12-29  0.903362  0.026116  1.192492
1125 2020-12-30  0.115565 -0.140657  0.474872
1126 2020-12-31  0.755455  0.383623  1.356276

[1127 rows x 4 columns]


In [122]:
import pandas as pd

# Assuming daily_mean_df is your DataFrame
daily_mean_df['time'] = pd.to_datetime(daily_mean_df['time'], format='%Y-%m-%d')

# Define the DJF period for 2017-2018
djf_dataset1 = daily_mean_df[((daily_mean_df['time'].dt.month == 12) & (daily_mean_df['time'].dt.year == 2017)) | 
                             ((daily_mean_df['time'].dt.month.isin([1, 2])) & (daily_mean_df['time'].dt.year == 2018))]

djf_dataset1.to_csv('djf_dataset1.csv', index=False)

# Display the sliced dataset
print(djf_dataset1)


         time       u10       v10  longitude  latitude    ws_avg
0  2017-12-01  0.780274  0.523236      110.0       0.0  1.379070
1  2017-12-02  0.608865  0.900866      110.0       0.0  1.446645
2  2017-12-03  0.373808  0.317415      110.0       0.0  0.885489
3  2017-12-04  0.349532 -0.079821      110.0       0.0  1.150266
4  2017-12-05  0.085431 -0.357639      110.0       0.0  0.944595
..        ...       ...       ...        ...       ...       ...
85 2018-02-24 -0.159274  0.132263      110.0       0.0  0.847453
86 2018-02-25 -0.177575 -0.301887      110.0       0.0  1.145544
87 2018-02-26 -0.144442 -0.067462      110.0       0.0  0.789686
88 2018-02-27  0.258488  0.284935      110.0       0.0  0.940875
89 2018-02-28  0.058288  0.292503      110.0       0.0  0.838229

[90 rows x 6 columns]


In [123]:
import xarray as xr
import pandas as pd
import numpy as np

# Assuming djf_dataset is already loaded as an xarray Dataset
# Calculate the distance from each station to the point (110, 0)
def haversine(lon1, lat1, lon2, lat2):
    # Calculate the great circle distance in kilometers between two points 
    # on the earth (specified in decimal degrees)
    # Source: https://stackoverflow.com/a/4913653/13090009
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371  # Radius of earth in kilometers
    return c * r

# Extract station coordinates
station_lons = djf_dataset['lon'].values
station_lats = djf_dataset['lat'].values

# Calculate distances
distances = haversine(110, 0, station_lons, station_lats)

# Find indices of the nearest stations (e.g., nearest 5 stations)
nearest_indices = np.argsort(distances)[:5]

# Slice the dataset to get the nearest stations
nearest_stations = djf_dataset.isel(Stasiun=nearest_indices)

# Convert to DataFrame for the relevant variables and reset index to flatten the table
df = nearest_stations[['ID Stasiun', 'Nama Stasiun', 'Provinsi', 'lat', 'lon', 'ws_avg', 'ws_max', 'ws_avg nonlts', 'ws_max nonlts', 'ltsmask', 'r2']].to_dataframe().reset_index()

df.to_csv('stasiunterdkt.csv', index=False)
# Display the DataFrame
print(df)


      Stasiun    Tanggal ID Stasiun             Nama Stasiun    Provinsi  \
0    14063042 2017-12-01   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
1    14063042 2017-12-02   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
2    14063042 2017-12-03   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
3    14063042 2017-12-04   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
4    14063042 2017-12-05   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
..        ...        ...        ...                      ...         ...   
445   STA2084 2018-02-24    STA2084             AWS Losarang  Jawa Barat   
446   STA2084 2018-02-25    STA2084             AWS Losarang  Jawa Barat   
447   STA2084 2018-02-26    STA2084             AWS Losarang  Jawa Barat   
448   STA2084 2018-02-27    STA2084             AWS Losarang  Jawa Barat   
449   STA2084 2018-02-28    STA2084             AWS Losarang  Jawa Barat   

          lat        lon    ws_avg  ws_max  ws_avg nonlts  ws_max nonlts  \
0   -6.3421

In [124]:
# Filter the DataFrame for the station with name '14063042'
station_14063042 = df[df['Stasiun'] == '14063042']

station_14063042.to_csv('station_14063042.csv', index=False)

# Display the DataFrame for the specific station
print(station_14063042)

     Stasiun    Tanggal ID Stasiun             Nama Stasiun    Provinsi  \
0   14063042 2017-12-01   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
1   14063042 2017-12-02   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
2   14063042 2017-12-03   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
3   14063042 2017-12-04   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
4   14063042 2017-12-05   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
..       ...        ...        ...                      ...         ...   
85  14063042 2018-02-24   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
86  14063042 2018-02-25   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
87  14063042 2018-02-26   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
88  14063042 2018-02-27   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   
89  14063042 2018-02-28   14063042  AAWS Indramayu 2 (AWS )  Jawa Barat   

         lat       lon    ws_avg  ws_max  ws_avg nonlts  ws_max nonlts  \
0  -6.342134  108.3413   

In [125]:
# Filter the DataFrame for the station with name '14063042'
station_sta2084 = df[df['Stasiun'] == 'STA2084']

station_sta2084.to_csv('station_sta2084.csv', index=False)

# Display the DataFrame for the specific station
print(station_sta2084)

     Stasiun    Tanggal ID Stasiun  Nama Stasiun    Provinsi      lat  \
360  STA2084 2017-12-01    STA2084  AWS Losarang  Jawa Barat -6.42064   
361  STA2084 2017-12-02    STA2084  AWS Losarang  Jawa Barat -6.42064   
362  STA2084 2017-12-03    STA2084  AWS Losarang  Jawa Barat -6.42064   
363  STA2084 2017-12-04    STA2084  AWS Losarang  Jawa Barat -6.42064   
364  STA2084 2017-12-05    STA2084  AWS Losarang  Jawa Barat -6.42064   
..       ...        ...        ...           ...         ...      ...   
445  STA2084 2018-02-24    STA2084  AWS Losarang  Jawa Barat -6.42064   
446  STA2084 2018-02-25    STA2084  AWS Losarang  Jawa Barat -6.42064   
447  STA2084 2018-02-26    STA2084  AWS Losarang  Jawa Barat -6.42064   
448  STA2084 2018-02-27    STA2084  AWS Losarang  Jawa Barat -6.42064   
449  STA2084 2018-02-28    STA2084  AWS Losarang  Jawa Barat -6.42064   

           lon    ws_avg  ws_max  ws_avg nonlts  ws_max nonlts  ltsmask  \
360  108.16681       NaN     NaN            NaN 

In [126]:
station_14063042['Tanggal'] = pd.to_datetime(station_14063042['Tanggal'], format='%Y-%m-%d')
station_sta2084['Tanggal'] = pd.to_datetime(station_sta2084['Tanggal'], format='%Y-%m-%d')

C:\Users\ahlil\AppData\Local\Temp\ipykernel_26932\3510631156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_14063042['Tanggal'] = pd.to_datetime(station_14063042['Tanggal'], format='%Y-%m-%d')
C:\Users\ahlil\AppData\Local\Temp\ipykernel_26932\3510631156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_sta2084['Tanggal'] = pd.to_datetime(station_sta2084['Tanggal'], format='%Y-%m-%d')


In [127]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# Drop rows with missing ws_avg values in station datasets
station_14063042 = station_14063042.dropna(subset=['ws_avg'])
station_sta2084 = station_sta2084.dropna(subset=['ws_avg'])


station_sta2084

,Stasiun,Tanggal,ID Stasiun,Nama Stasiun,Provinsi,lat,lon,ws_avg,ws_max,ws_avg nonlts,ws_max nonlts,ltsmask,r2
376,STA2084,2017-12-17,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,3.624590,12.100000,3.624590,12.100000,1.0,0.982344
377,STA2084,2017-12-18,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,4.657752,16.100000,4.657752,16.100000,1.0,0.982344
378,STA2084,2017-12-19,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,2.673246,12.880000,2.673246,12.880000,1.0,0.982344
382,STA2084,2017-12-23,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.409132,8.300000,1.409132,8.300000,1.0,0.982344
383,STA2084,2017-12-24,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.467178,8.400001,1.467178,8.400001,1.0,0.982344
384,STA2084,2017-12-25,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.849088,8.520000,1.849088,8.520000,1.0,0.982344
386,STA2084,2017-12-27,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,4.218455,10.950000,4.218455,10.950000,1.0,0.982344
393,STA2084,2018-01-03,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.219208,8.430000,1.219208,8.430000,1.0,0.982344
394,STA2084,2018-01-04,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.384935,8.960000,1.384935,8.960000,1.0,0.982344
399,STA2084,2018-01-09,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,2.459035,15.680000,2.459035,15.680000,1.0,0.982344


In [128]:
import pandas as pd

# Read the Excel file without using any column as the index
bmkg = pd.read_excel('bmkg.xlsx', index_col=None)

# Convert the 'Tanggal' column to datetime format
bmkg['Tanggal'] = pd.to_datetime(bmkg['Tanggal'],format = '%d-%m-%Y')

# Display the first few rows of the DataFrame
bmkg


,Tanggal,ws_avg
0,2017-01-12,4.0
1,2017-02-12,4.0
2,2017-03-12,4.0
3,2017-04-12,5.0
4,2017-05-12,4.0
...,...,...
85,2018-02-24,2.0
86,2018-02-25,1.0
87,2018-02-26,1.0
88,2018-02-27,2.0


In [136]:
# Rename the 'time' column to 'Tanggal'
djf_dataset1.rename(columns={'time': 'Tanggal'}, inplace=True)
djf_dataset1

C:\Users\ahlil\AppData\Local\Temp\ipykernel_26932\1328981710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  djf_dataset1.rename(columns={'time': 'Tanggal'}, inplace=True)


,Tanggal,u10,v10,longitude,latitude,ws_avg
18,2017-12-19,0.479949,-1.159938,110.0,0.0,1.360757
20,2017-12-21,1.198068,-1.040137,110.0,0.0,1.691244
22,2017-12-23,1.286288,0.246591,110.0,0.0,1.417665
24,2017-12-25,1.008769,0.688927,110.0,0.0,1.370527
25,2017-12-26,0.290061,-0.997969,110.0,0.0,1.170778
...,...,...,...,...,...,...
85,2018-02-24,-0.159274,0.132263,110.0,0.0,0.847453
86,2018-02-25,-0.177575,-0.301887,110.0,0.0,1.145544
87,2018-02-26,-0.144442,-0.067462,110.0,0.0,0.789686
88,2018-02-27,0.258488,0.284935,110.0,0.0,0.940875


In [130]:
# Convert the 'Tanggal' columns to datetime
station_14063042['Tanggal'] = pd.to_datetime(station_14063042['Tanggal'])
station_sta2084['Tanggal'] = pd.to_datetime(station_sta2084['Tanggal'])

# Extract the unique dates from the station data
station_dates = station_14063042['Tanggal'].unique()

# Filter the BMKG data to only include rows with dates in the station data
station_sta2084 = station_sta2084[station_sta2084['Tanggal'].isin(station_dates)]
station_sta2084


,Stasiun,Tanggal,ID Stasiun,Nama Stasiun,Provinsi,lat,lon,ws_avg,ws_max,ws_avg nonlts,ws_max nonlts,ltsmask,r2
378,STA2084,2017-12-19,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,2.673246,12.88,2.673246,12.88,1.0,0.982344
382,STA2084,2017-12-23,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.409132,8.30,1.409132,8.30,1.0,0.982344
384,STA2084,2017-12-25,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.849088,8.52,1.849088,8.52,1.0,0.982344
386,STA2084,2017-12-27,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,4.218455,10.95,4.218455,10.95,1.0,0.982344
393,STA2084,2018-01-03,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.219208,8.43,1.219208,8.43,1.0,0.982344
394,STA2084,2018-01-04,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,1.384935,8.96,1.384935,8.96,1.0,0.982344
399,STA2084,2018-01-09,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,2.459035,15.68,2.459035,15.68,1.0,0.982344
400,STA2084,2018-01-10,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,3.903937,14.08,3.903937,14.08,1.0,0.982344
401,STA2084,2018-01-11,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,4.966455,15.59,4.966455,15.59,1.0,0.982344
402,STA2084,2018-01-12,STA2084,AWS Losarang,Jawa Barat,-6.42064,108.16681,2.843383,12.84,2.843383,12.84,1.0,0.982344


In [131]:
# Convert the 'Tanggal' columns to datetime
station_14063042['Tanggal'] = pd.to_datetime(station_14063042['Tanggal'])
djf_dataset1['time'] = pd.to_datetime(djf_dataset1['time'])

# Extract the unique dates from the station data
station_dates = station_14063042['Tanggal'].unique()

# Filter the BMKG data to only include rows with dates in the station data
djf_dataset1 = djf_dataset1[djf_dataset1['time'].isin(station_dates)]
djf_dataset1


C:\Users\ahlil\AppData\Local\Temp\ipykernel_26932\175027372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  djf_dataset1['time'] = pd.to_datetime(djf_dataset1['time'])


,time,u10,v10,longitude,latitude,ws_avg
18,2017-12-19,0.479949,-1.159938,110.0,0.0,1.360757
20,2017-12-21,1.198068,-1.040137,110.0,0.0,1.691244
22,2017-12-23,1.286288,0.246591,110.0,0.0,1.417665
24,2017-12-25,1.008769,0.688927,110.0,0.0,1.370527
25,2017-12-26,0.290061,-0.997969,110.0,0.0,1.170778
...,...,...,...,...,...,...
85,2018-02-24,-0.159274,0.132263,110.0,0.0,0.847453
86,2018-02-25,-0.177575,-0.301887,110.0,0.0,1.145544
87,2018-02-26,-0.144442,-0.067462,110.0,0.0,0.789686
88,2018-02-27,0.258488,0.284935,110.0,0.0,0.940875


In [132]:
# Convert the 'Tanggal' columns to datetime
station_14063042['Tanggal'] = pd.to_datetime(station_14063042['Tanggal'])
bmkg['Tanggal'] = pd.to_datetime(bmkg['Tanggal'])

# Extract the unique dates from the station data
station_dates = station_14063042['Tanggal'].unique()

# Filter the BMKG data to only include rows with dates in the station data
bmkg = bmkg[bmkg['Tanggal'].isin(station_dates)]
bmkg



,Tanggal,ws_avg
18,2017-12-19,2.0
20,2017-12-21,3.0
22,2017-12-23,2.0
24,2017-12-25,4.0
25,2017-12-26,3.0
26,2017-12-27,3.0
31,2018-01-01,4.0
32,2018-02-01,2.0
44,2018-01-14,3.0
45,2018-01-15,3.0


In [137]:
# Merge datasets on 'Tanggal' column
merged_bmkg = pd.merge(djf_dataset1, bmkg, on='Tanggal', suffixes=('_djf', '_bmkg'))
merged_station_14063042 = pd.merge(djf_dataset1, station_14063042, on='Tanggal', suffixes=('_djf', '_14063042'))
merged_station_sta2084 = pd.merge(djf_dataset1, station_sta2084, on='Tanggal', suffixes=('_djf', '_sta2084'))

In [138]:
# Calculate RMSE
def calculate_rmse(merged_df, col1, col2):
    merged_df = merged_df.dropna(subset=[col1, col2])
    return np.sqrt(((merged_df[col1] - merged_df[col2]) ** 2).mean())

rmse_bmkg = calculate_rmse(merged_bmkg, 'ws_avg_djf', 'ws_avg_bmkg')
rmse_14063042 = calculate_rmse(merged_station_14063042, 'ws_avg_djf', 'ws_avg_14063042')
rmse_sta2084 = calculate_rmse(merged_station_sta2084, 'ws_avg_djf', 'ws_avg_sta2084')

# Calculate correlation
correlation_bmkg = merged_bmkg['ws_avg_djf'].corr(merged_bmkg['ws_avg_bmkg'])
correlation_14063042 = merged_station_14063042['ws_avg_djf'].corr(merged_station_14063042['ws_avg_14063042'])
correlation_sta2084 = merged_station_sta2084['ws_avg_djf'].corr(merged_station_sta2084['ws_avg_sta2084'])

# Print the results
print("RMSE with BMKG:", rmse_bmkg)
print("Correlation with BMKG:", correlation_bmkg)
print("RMSE with Station 14063042:", rmse_14063042)
print("Correlation with Station 14063042:", correlation_14063042)
print("RMSE with Station sta2084:", rmse_sta2084)
print("Correlation with Station sta2084:", correlation_sta2084)

RMSE with BMKG: 1.5469835998765262
Correlation with BMKG: 0.5751470949809488
RMSE with Station 14063042: 1.0609952345230127
Correlation with Station 14063042: 0.5370151143759657
RMSE with Station sta2084: 2.1393587382116244
Correlation with Station sta2084: 0.5771172407019742


In [139]:
# Calculate RMSE
def calculate_rmse(merged_df, col1, col2):
    merged_df = merged_df.dropna(subset=[col1, col2])
    return np.sqrt(((merged_df[col1] - merged_df[col2]) ** 2).mean())

# Calculate Bias
def calculate_bias(merged_df, col1, col2):
    merged_df = merged_df.dropna(subset=[col1, col2])
    return (merged_df[col1] - merged_df[col2]).mean()

rmse_bmkg = calculate_rmse(merged_bmkg, 'ws_avg_djf', 'ws_avg_bmkg')
rmse_14063042 = calculate_rmse(merged_station_14063042, 'ws_avg_djf', 'ws_avg_14063042')
rmse_sta2084 = calculate_rmse(merged_station_sta2084, 'ws_avg_djf', 'ws_avg_sta2084')

bias_bmkg = calculate_bias(merged_bmkg, 'ws_avg_djf', 'ws_avg_bmkg')
bias_14063042 = calculate_bias(merged_station_14063042, 'ws_avg_djf', 'ws_avg_14063042')
bias_sta2084 = calculate_bias(merged_station_sta2084, 'ws_avg_djf', 'ws_avg_sta2084')

# Calculate correlation
correlation_bmkg = merged_bmkg['ws_avg_djf'].corr(merged_bmkg['ws_avg_bmkg'])
correlation_14063042 = merged_station_14063042['ws_avg_djf'].corr(merged_station_14063042['ws_avg_14063042'])
correlation_sta2084 = merged_station_sta2084['ws_avg_djf'].corr(merged_station_sta2084['ws_avg_sta2084'])

# Create tables
rmse_table = pd.DataFrame({
    'Dataset': ['BMKG', 'Station 14063042', 'Station sta2084'],
    'RMSE': [rmse_bmkg, rmse_14063042, rmse_sta2084]
})

correlation_table = pd.DataFrame({
    'Dataset': ['BMKG', 'Station 14063042', 'Station sta2084'],
    'Correlation': [correlation_bmkg, correlation_14063042, correlation_sta2084]
})

bias_table = pd.DataFrame({
    'Dataset': ['BMKG', 'Station 14063042', 'Station sta2084'],
    'Bias': [bias_bmkg, bias_14063042, bias_sta2084]
})

# Display the tables
print("RMSE Table:")
print(rmse_table)

print("\nCorrelation Table:")
print(correlation_table)

print("\nBias Table:")
print(bias_table)

RMSE Table:
            Dataset      RMSE
0              BMKG  1.546984
1  Station 14063042  1.060995
2   Station sta2084  2.139359

Correlation Table:
            Dataset  Correlation
0              BMKG     0.575147
1  Station 14063042     0.537015
2   Station sta2084     0.577117

Bias Table:
            Dataset      Bias
0              BMKG -1.251876
1  Station 14063042 -0.884682
2   Station sta2084 -1.934006
